In [1]:
import findspark

In [3]:
findspark.init('/home/ky/spark-3.0.1-bin-hadoop3.2')

# Just in case to find pysparklib

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('Stocks').getOrCreate()

In [45]:
df = spark.read.csv('dataframe/WMT.csv', header= True, inferSchema= True)

In [46]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Adj Close: double (nullable = true)
 |-- Volume: integer (nullable = true)



In [47]:
df.show()

+----------+----------+----------+----------+----------+----------+--------+
|      Date|      Open|      High|       Low|     Close| Adj Close|  Volume|
+----------+----------+----------+----------+----------+----------+--------+
|2020-02-14|117.669998|    118.57|117.160004|117.889999| 115.95433| 8130600|
|2020-02-18|118.470001|119.949997|117.360001|119.629997|117.665764|11513200|
|2020-02-19|     119.5|119.940002|    117.68|    117.68| 115.74778| 7187800|
|2020-02-20|117.209999|118.110001|116.860001|117.690002|115.757614| 5022900|
|2020-02-21|117.440002|    118.75|117.309998|118.580002|116.633011| 6242100|
|2020-02-24|117.459999|118.459999|115.949997|    116.32| 114.41011| 6616000|
|2020-02-25|116.349998|    117.07|114.050003|114.389999|112.511795| 7764300|
|2020-02-26|114.699997|115.169998|113.650002|113.779999|111.911819| 6673300|
|2020-02-27|112.559998|    114.07|110.349998|110.400002|108.587311| 9492300|
|2020-02-28|107.690002|108.519997|104.370003|    107.68|105.911972|17504100|

## To get Colums names

In [48]:
df.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']

## To get first 5 rows

In [49]:
df.limit(5).collect()

[Row(Date='2020-02-14', Open=117.669998, High=118.57, Low=117.160004, Close=117.889999, Adj Close=115.95433, Volume=8130600),
 Row(Date='2020-02-18', Open=118.470001, High=119.949997, Low=117.360001, Close=119.629997, Adj Close=117.665764, Volume=11513200),
 Row(Date='2020-02-19', Open=119.5, High=119.940002, Low=117.68, Close=117.68, Adj Close=115.74778, Volume=7187800),
 Row(Date='2020-02-20', Open=117.209999, High=118.110001, Low=116.860001, Close=117.690002, Adj Close=115.757614, Volume=5022900),
 Row(Date='2020-02-21', Open=117.440002, High=118.75, Low=117.309998, Close=118.580002, Adj Close=116.633011, Volume=6242100)]

In [50]:
df.describe().show()

+-------+----------+-----------------+------------------+------------------+------------------+------------------+-----------------+
|summary|      Date|             Open|              High|               Low|             Close|         Adj Close|           Volume|
+-------+----------+-----------------+------------------+------------------+------------------+------------------+-----------------+
|  count|       252|              252|               252|               252|               252|               252|              252|
|   mean|      null|     132.89829375|134.42234131349198|131.47805582936505|132.96301598015867| 132.1596568888889|9244117.857142856|
| stddev|      null|12.07134104498652|11.633165123345597|12.278032457203182| 11.85547736640773|12.319401872389507| 4970391.33355953|
|    min|2020-02-14|       105.199997|        108.519997|             102.0|        104.050003|        102.341583|          3018200|
|    max|2021-02-12|       153.600006|        153.660004|        151.

In [51]:
from pyspark.sql.functions import format_number

In [52]:
## To get two decimal val from mean, stddev(standard deviation)

In [53]:
result = df.describe()

In [55]:
result.select(result['summary'],format_number(result['Open'].cast('float'), 2).alias('Open')).show()

+-------+------+
|summary|  Open|
+-------+------+
|  count|252.00|
|   mean|132.90|
| stddev| 12.07|
|    min|105.20|
|    max|153.60|
+-------+------+



In [60]:
# a result to get HV ratio means result of high and volume ratio 

In [62]:
df2 = df.withColumn('HV Ratio', df['High']/df['Volume'])

In [74]:
df2.select(df2['HV Ratio']).show()

+--------------------+
|            HV Ratio|
+--------------------+
|1.458317959314195...|
|1.041847592328805...|
|1.668660814157322E-5|
|2.351430468454478...|
|1.902404639464283E-5|
|1.790507844619105E-5|
|1.507798513710186...|
|1.725832766397434...|
|1.201710860381572...|
|6.199690186870505E-6|
|6.817303416851156E-6|
|1.057533103584797...|
|1.423419862579796...|
|1.371970840500345...|
|1.202709222028418E-5|
|6.108528393036186E-6|
| 9.57846195363161E-6|
|1.129316878557874...|
|5.938405381796488...|
|8.001703548010921E-6|
+--------------------+
only showing top 20 rows



In [75]:
# get date with peak high in price

In [78]:
df.orderBy(df['High'].desc()).head(1)[0][0]

'2020-12-01'

In [96]:
from pyspark.sql.functions import mean, max, min

In [97]:
df.select(mean('Close')).show()

+------------------+
|        avg(Close)|
+------------------+
|132.96301598015867|
+------------------+



In [99]:
df.select(min('Close'), max('Close')).show()

+----------+----------+
|min(Close)|max(Close)|
+----------+----------+
|104.050003|152.789993|
+----------+----------+



In [103]:
## days was the close lower than 200 using filter

In [112]:
df.filter('Close < 200').count()

252

In [114]:
df.select(df['Close'] < 200).count()

252

In [116]:
df.select(df['High'] > 80).count()/df.count()

1.0

In [151]:
from pyspark.sql.functions import corr, year, avg, month

In [152]:
df.select(corr('High', 'Volume')).show()

+--------------------+
|  corr(High, Volume)|
+--------------------+
|-0.19335095966742472|
+--------------------+



In [153]:
yeardf = df.withColumn('Year', year(df['Date']))

In [164]:
yeardf.groupBy('Year').max('High').show()

+----+----------+
|Year| max(High)|
+----+----------+
|2020|153.660004|
|2021|149.929993|
+----+----------+



In [169]:
monthdf = df.withColumn('Month', month(df['Date']))

In [172]:
monthdf.groupBy('Month').avg('Close').show()

+-----+------------------+
|Month|        avg(Close)|
+-----+------------------+
|   12|146.57681759090912|
|    1|145.75842042105262|
|    6|120.50636331818183|
|    3|114.41727345454545|
|    5|124.07350000000001|
|    9|138.90714223809525|
|    4|125.78809571428572|
|    8|132.46905080952382|
|    7|128.98181804545453|
|   10|142.59499986363633|
|   11|148.09999845000002|
|    2|       129.2825008|
+-----+------------------+

